# Preprocess Wikipedia with Spark

## TL/DR:

1. Start with a Wikipedia Dump
2. Process the data in Spark.
3. Produce maps like this: ![alt text](all_wikipedia_coords.png "Title")

Resources:
* https://en.wikipedia.org/wiki/Wikipedia:Database_download
* https://towardsdatascience.com/wikipedia-data-science-working-with-the-worlds-largest-encyclopedia-c08efbac5f5c
* https://github.com/earwig/mwparserfromhell

## Install dependencies

This is using `%pip` rather than `pkg_rsources.resolve()` because on databricks clusters, `%pip` will make sure the libraries are available on the spark worker nodes.

In [ ]:
required_packages = {"mwparserfromhell","geopandas","h3","geocoder","pydeck"}

import pkg_resources
for lib in required_packages - {pkg.key for pkg in pkg_resources.working_set}:
    print(f"installing {lib}")
    %pip install -q --upgrade pip
    %pip install -q $lib
    pkg_resources.require(lib)


In [ ]:
# Where to find (and if needed download) a wikipedia dump file
source_dir      = './data/wikipedia_src'
source_dump     = source_dir + '/enwiki-latest-pages-articles-multistream.xml.bz2'

# Where to store temporary pre-processed files before creating spark tables.  Will temporarily use ~70GB.
preprocess_dir  = './data/wikipedia_as_json'


In [7]:
import xml.sax
import json
import mwparserfromhell
import subprocess
import json
import time
import IPython

## Fetch the latest Wikipedia dump

In [ ]:
! mkdir -p data/wikipedia_src data/wikipedia_as_json
! (cd data/wikipedia_src; wget -nc https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles-multistream.xml.bz2)

## Pre-process the Wikimedia XML to JSONL / NLJSON

* Spark's tooling for JSONL is better than it's XML handling so pre-process the XML to JSONL.

* Put 100,000 Wikipeda pages in each .json file.
    * JSON files each containing 10,000 to 100,000 Wikipedia pages seems to be a sweet spot for loading as Spark dataframes on this cluster.
    * This generates about 215 files, averaging 325MB each.

* Yes, this preprocessing step could be further optimized, but it doesn't matter.
    * The XML->JSON conversion is fast enough it doesn't need to be parallized. It takes under 2 hours on a single CPU core.  In contrast, parsing the wikimedia markup below takes many hours on a cluster of 32 CPUs.  Compressing the temp files could save space, but it doesn't matter either.


In [5]:
# Based heavily on
# https://github.com/WillKoehrsen/wikipedia-data-science/blob/master/notebooks/Downloading%20and%20Parsing%20Wikipedia%20Articles.ipynb

class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []
        self._page_count = 0

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'timestamp'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)
        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))
            
    def save_partial_results(self,path):
        self._page_count += len(handler._pages)
        filename = f'{path}/{self._page_count}.jsonl'
        with open(filename,'w') as jsonfile:
            for page in handler._pages:
                page_dict = {'title':page[0],'body':page[1]}
                json.dump(page_dict,jsonfile)
                jsonfile.write("\n")
        self._pages = []
      

In [6]:
rebuild_json_files = False
if rebuild_json_files:
    wikipedia_dump = source_dump
    handler        = WikiXmlHandler()
    parser         = xml.sax.make_parser()
    parser.setContentHandler(handler)
    expected_pages = 21100000  # Doesn't need to be exact. Just used to show progress.
    t0 = time.time()
    for i, line in enumerate(subprocess.Popen(['bzcat'], 
                             stdin = open(wikipedia_dump), 
                             stdout = subprocess.PIPE).stdout):
        parser.feed(line)
        if len(handler._pages) >= 100000:
            handler.save_partial_results(preprocess_dir)
            t1 = time.time()
            et = t1 - t0
            page_count = handler._page_count
            print(f'page {page_count} at {int(et)} seconds '
                  f'({int(page_count/(et))} per second). '
                  f'Estimate {int(et*expected_pages/page_count/60)} minutes '
                  f'{et*(expected_pages-page_count)/page_count/60} remaining'
                 )
    handler.save_partial_results(preprocess_dir)

## Launch Spark (if running on a standalone environment)

* On databricks clusters the Spark Context will already have existed.

In [2]:
if not "spark" in locals():
    import pyspark
    MAX_MEMORY = "8g"  # 24 gives OOM here. # 6 gives "out of heap space"
    spark = (pyspark.sql.SparkSession.builder.appName("MyApp") 
        .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") 
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
        .config("spark.executor.memory", MAX_MEMORY) 
        .config("spark.driver.memory", MAX_MEMORY) 
        .config("spark.python.worker.reuse",False)
        .config("spark.task.maxFailures",5)
        .enableHiveSupport() 
        .getOrCreate()        
        )
spark

In [8]:
rebuild_delta_table = False
if rebuild_delta_table:
    t0 = time.perf_counter()
    df = spark.read.json(preprocess_dir,'title string, body string')
    df.select('title','body').write.format('delta').saveAsTable('wikipedia_bronze')
    print(f"saved as delta table in {time.perf_counter() - t0} seconds")
    napier_df = spark.sql("select * from wikipedia_bronze where title = 'Charles James Napier' limit 3").cache()
    napier_df.write.format('delta').saveAsTable('wikipedia_bronze_small')

In [11]:
spark.sql(r"select title,replace(substr(body,1,40)||'...','\n','\\n') as body from wikipedia_bronze limit 5").show(5,False)

+-----------------------------+---------------------------------------------+
|title                        |body                                         |
+-----------------------------+---------------------------------------------+
|Black panther                |{{short description|Melanistic colour va...  |
|Great white shark            |{{other uses of|great white|Great White ...  |
|Melisende, Queen of Jerusalem|{{short description|Queen regnant of the...  |
|Carcharodon carcharias       |#REDIRECT [[Great white shark]] \n \n {{Re...|
|Method                       |{{Wiktionary|method}} \n '''Method''' ({{... |
+-----------------------------+---------------------------------------------+



## Parse the wikipedia content

* Wraps the excellent but pretty slow `mwparserfromhell` library: https://github.com/earwig/mwparserfromhell

* Attempts to have this Pandas UDF return comples structures like 

      infoboxes array<struct<name:string,params:map<string,string>,body:string>>,
      templates array<struct<name:string,params:map<string,string>,body:string>>,
      extlinks  array<struct<title:string,url:string>>,
      wikilinks array<struct<text:string,title:string>>,
      _error_   string
      
    are failing with the following in Spark 3.1.1:

        21/04/15 07:05:29 WARN BlockManager: Putting block rdd_66_0 failed due to exception org.apache.spark.TaskKilledException.
        21/04/15 07:05:29 ERROR TaskContextImpl: Error in TaskCompletionListener
        java.lang.IllegalStateException: Memory was leaked by query. Memory leaked: (4194304)
        Allocator(stdin reader for python3) 0/4194304/12602208/9223372036854775807 (res/actual/peak/limit)
        at org.apache.arrow.memory.BaseAllocator.close(BaseAllocator.java:431)
    
* Work around it by returning the data encoded as a json string.    
  Doesn't add much overhead compared to the actual parsing.

In [12]:
import pyspark.sql.functions as psf
import pandas as pd
import re
import json
import h3 as h3

def all_h3_grids(lat,lon):
    return([h3.geo_to_h3(lat, lon, lvl)  for lvl in range(16)])

# https://en.wikipedia.org/wiki/Template:Coord
#
# {{coord|latitude|longitude|coordinate parameters|template parameters}}
# {{coord|dd|N/S|dd|E/W|coordinate parameters|template parameters}}
# {{coord|dd|mm|N/S|dd|mm|E/W|coordinate parameters|template parameters}}
# {{coord|dd|mm|ss|N/S|dd|mm|ss|E/W|coordinate parameters|template parameters}}
#
def interpret_wikimedia_coordinate(parsed_template):
    lat = lon = None
    params = {p.name.strip(): p.value.strip() for p in parsed_template.params}
    if not (params.get('4') in ['N','S']
            or 
            params.get('3') in ['N','S']
            or 
            params.get('2') in ['N','S']
           ):
        lat = float(params['1'])
        lon = float(params['2'])
    if params.get('4') == 'N': lat =    int(params['1']) + float(params['2'])/60 + float(params['3'])/60/60
    if params.get('4') == 'S': lat = - (int(params['1']) + float(params['2'])/60 + float(params['3'])/60/60)
    if params.get('8') == 'E': lon =    int(params['5']) + float(params['6'])/60 + float(params['7'])/60/60
    if params.get('8') == 'W': lon = - (int(params['5']) + float(params['6'])/60 + float(params['7'])/60/60)
    if params.get('3') == 'N': lat =    int(params['1']) + float(params['2'])/60 
    if params.get('3') == 'S': lat = - (int(params['1']) + float(params['2'])/60 )
    if params.get('6') == 'E': lon =    int(params['4']) + float(params['5'])/60 
    if params.get('6') == 'W': lon = - (int(params['4']) + float(params['5'])/60 )
    if params.get('2') == 'N': lat =    float(params['1'])
    if params.get('2') == 'S': lat = - (float(params['1']) )
    if params.get('4') == 'E': lon =    float(params['3'])
    if params.get('4') == 'W': lon = - (float(params['3']) )
    if lat and lon: return [lat,lon]
    return None

# https://en.wikipedia.org/wiki/Template:Birth_date
def extract_mediawiki_info(wikimedia_text):
    def nstr(s): 
        return(s and s.strip())
    def params_as_dict(tmpl):
        return({p.name.strip(): p.value.strip() for p in tmpl.params})
    def tmpl_as_dict(tmpl):
        return({"name":nstr(tmpl.name),"params":params_as_dict(tmpl),"body":nstr(tmpl)})
    try:
        p = mwparserfromhell.parse(wikimedia_text)
        all_tmpl    = p.filter_templates(recursive=True) 
        top_tmpl    = p.filter_templates(recursive=False)
        raw_coords  = [t for t in all_tmpl if t.name.strip().lower() == 'coord']
        lat_lon = [interpret_wikimedia_coordinate(coord) for coord in raw_coords if coord]
        coords  = [{'lat':coord[0],
                    'lon':coord[1],
                    'h3':all_h3_grids(coord[0],coord[1])
                  } for coord in lat_lon]
        templates   = [tmpl_as_dict(t) for t in top_tmpl]
        infoboxes   = [t for t in templates if     t['name'].lower().startswith('infobox') ]
        othertmpl   = [t for t in templates if not t['name'].lower().startswith('infobox') ]
        wikilinks   = [{"title":nstr(l.title),"text":nstr(l.text)} for l in p.filter_wikilinks()]
        extlinks    = [{"title":nstr(l.title),"url":nstr(l.url)}   for l in p.filter_external_links()]
        return json.dumps({
            "infoboxes":infoboxes,
            "coords":coords,
            "templates":othertmpl,
            "wikilinks":wikilinks,
            "extlinks":extlinks
        })
    except Exception as e:
        return json.dumps({"_error_":str(e)})

parsed_schema = '''
      infoboxes array<struct<name:string,params:map<string,string>,body:string>>,
      templates array<struct<name:string,params:map<string,string>,body:string>>,
      extlinks  array<struct<title:string,url:string>>,
      wikilinks array<struct<text:string,title:string>>,
      coords    array<struct<lat:float,lon:float,h3:array<string>>>,
      _error_   string
    '''

@psf.pandas_udf('string')
def extract_mediawiki_info_udf(s: pd.Series) -> pd.Series:
    return s.apply(extract_mediawiki_info)
spark.udf.register('extract_mediawiki_info_udf',extract_mediawiki_info_udf)

def template_params(wikimedia_text):
    try:
        p = mwparserfromhell.parse(wikimedia_text)
        templates   = p.filter_templates(recursive=False)
        if len(templates) == 0: return None
        params = {p.name.strip(): p.value.strip() for p in templates[0].params}
        return params
    except Exception as e:
        return {}

@psf.pandas_udf('map<string,string>')
def template_params_udf(s: pd.Series) -> pd.Series:
    return s.apply(template_params)
spark.udf.register('template_params_udf',template_params_udf)

@psf.pandas_udf('array<float>')
def wikimedia_coordinates_udf(s: pd.Series) -> pd.Series:
    def wikimedia_coordinates(wikimedia_text):
        try:
            p = mwparserfromhell.parse(wikimedia_text)
            templates = [t for t in p.filter_templates(recursive=False) if t.name.lower() == 'coord']  
            if len(templates) == 0: return None
            return interpret_wikimedia_coordinate(templates[0])
        except Exception as e:
            return None
    return(s.apply(wikimedia_coordinates))
spark.udf.register('wikimedia_coordinates_udf',wikimedia_coordinates_udf)

@psf.pandas_udf('array<string>')
def all_h3_grids_udf(lat: pd.Series,lon: pd.Series) -> pd.Series:
    result = [ [h3.geo_to_h3(lat, lon, lvl)  for lvl in range(16)] for la,lo in zip(lat,lon)]
    print(result)
    return pd.Series(result)


In [13]:
test_mediawiki_parser_udf = True
if test_mediawiki_parser_udf:
    infoboxes_with_locations_df = spark.sql('''
        select * 
          from wikipedia_bronze 
          where lower(body) like '%{{infobox%' and lower(body) like '%{{coord%' 
          limit 5
    ''').cache()
    person_infoboxes_df = spark.sql('''
          select * from (select * 
            from wikipedia_bronze 
            where title in ('Hans Albert Einstein', 'Charles James Napier')
            limit 5) as person_infoboxes
    ''').cache()
    interesting_test_cases_df = infoboxes_with_locations_df.union(person_infoboxes_df)
    interesting_test_cases_df.createOrReplaceTempView('interesting_test_casess')
    print(f'parsing {interesting_test_cases_df.take(1)[0].title} in the driver')
    txt = interesting_test_cases_df.take(1)[0].body
    json.loads(extract_mediawiki_info(txt))
    print(f'parsing all {interesting_test_cases_df.count()} using the udf')
    df1 = interesting_test_cases_df
    df2 = df1.withColumn('parsed_elements',extract_mediawiki_info_udf('body'))
    df3 = df2.withColumn('parsed_elements',psf.from_json('parsed_elements',parsed_schema))
    df3.selectExpr('title',
                   'parsed_elements.coords',
                   "parsed_elements.infoboxes[0].params['birth_date'] as dob"
                  ).show(30,50)
    

parsing Frankie Yankovic in the driver
parsing all 7 using the udf
+------------------------------+--------------------------------------------------+-------------------------------+
|                         title|                                            coords|                            dob|
+------------------------------+--------------------------------------------------+-------------------------------+
|              Frankie Yankovic|[{41.569, -81.5752, [802bfffffffffff, 812abffff...|                           null|
|                        Veneto|[{45.733334, 11.85, [801ffffffffffff, 811ebffff...|                           null|
|                      Lombardy|[{45.585556, 9.930278, [801ffffffffffff, 811fbf...|                           null|
|             Odense University|[{55.369232, 10.428814, [801ffffffffffff, 811f3...|                           null|
|University of Southern Denmark|[{55.36861, 10.428056, [801ffffffffffff, 811f3f...|                           null|
|    

In [14]:
test_coords = True
if test_coords:
    # https://en.wikipedia.org/wiki/Template:Coord
    testcases = '''
        {{coord|43.651234|-79.383333}}     43.651234°N 79.383333°W    Toronto – Fully decimal – western hemisphere implied by negation
        {{coord|43.65|-79.38}}     43.65°N 79.38°W    Toronto – low precision decimal
        {{coord|43.6500|-79.3800}}     43.6500°N 79.3800°W     Toronto – medium precision decimal with trailing zeroes
        {{coord|43.653500|N|79.384000|W}}     43.653500°N 79.384000°W     Toronto – high precision decimal with explicit hemisphere notation
        {{coord|43|29|N|79|23|W}}     43°29′N 79°23′W     Toronto – degrees & minutes
        {{coord|43|29|13|N|79|23|02|W}}     43°29′4″N 79°23′0″W     Toronto – degrees, minutes & seconds
        {{coord|43|29|12.6|N|79|23|02.4|W}}     43°29′12.6″N 79°23′02.4″W     Toronto – degrees, minutes, seconds & fractions of seconds
        {{coord|55.752222|N|37.615556|E}}     55.752222°N 37.615556°E     Moscow – N & E
        {{coord|55.752222|N|37.615556|E|format=dms}}     55°45′08″N 37°36′56″E     Convert to dms format
        {{coord|39.098095|-94.587307|format=dms}}     39°05′53″N 94°35′14″W     Decimal conversion without N/S/E/W
        {{coord|55.752222|N|37.615556|E|format=dec|name=Moscow}}     55.752222°N 37.615556°E     Convert to decimal and label on some maps
        {{coord|33|55|S|18|25|E}}     33°55′S 18°25′E     Cape Town – S & E
        {{coord|35|00|N|105|00|E}}     35°00′N 105°00′E     People's Republic of China
        {{coord|22|54|30|S|43|14|37|W}}     22°54′30″S 43°14′37″W     Rio – S & W
        {{coord|22|S|43|W}}     22°S 43°W     A degree confluence.
        {{coord|52|28|N|1|55|W|region:GB_type:city|notes=<ref>{{cite web|url=http://www.fallingrain.com/world/UK/0/Birmingham.html|title=Birmingham}}</ref>|display=inline,title}}     52°28′N 1°55′W[1]Coordinates: 52°28′N 1°55′W[1]     Birmingham – with display, notes, and parameter settings; note that these coordinates are also displayed at the top of this page.
        {{coord|51|25.813|N|0|43.945|E}}     51°25.813′N 0°43.945′E     Navigation buoy in the River Medway, England.
        {{coord|51|36.287|N|8|32.018|W}}     51°36.287′N 8°32.018′W     Lighthouse at the Old Head of Kinsale as defined by the Commissioners of Irish Lights. 
    '''.splitlines()
    spark.createDataFrame([[testcase] for testcase in testcases],'testcase string').createOrReplaceTempView('coord_testcases')
    spark.sql('''select wikimedia_coordinates_udf(testcase) as latlon,substr(testcase,1,90)||'...' as testcase from coord_testcases''').show(40,90)


+-----------------------+------------------------------------------------------------------------------------------+
|                 latlon|                                                                                  testcase|
+-----------------------+------------------------------------------------------------------------------------------+
|                   null|                                                                                       ...|
| [43.651234, -79.38333]|        {{coord|43.651234|-79.383333}}     43.651234°N 79.383333°W    Toronto – Fully d...|
|        [43.65, -79.38]|          {{coord|43.65|-79.38}}     43.65°N 79.38°W    Toronto – low precision decimal...|
|        [43.65, -79.38]|        {{coord|43.6500|-79.3800}}     43.6500°N 79.3800°W     Toronto – medium precisi...|
|     [43.6535, -79.384]|        {{coord|43.653500|N|79.384000|W}}     43.653500°N 79.384000°W     Toronto – hig...|
| [43.483334, -79.38333]|          {{coord|43|29|N|79|23|W}}    

#### Apply the mwparserfromhell parser on each document.

* Work around memory-related limitations on tiny/cheap clusters (Azure's cheapest tier of Databricks workers, or a local desktop).

The default`spark.sql.execution.arrow.maxRecordsPerBatch` of 10,000 doesn't work well when you hit ranges of some of the larger documents like

    {
       "title": "Wikipedia:Arbitration Committee Elections December 2018/Coordination/Mass message",
       "length(body)": 4483016
    },
    {
       "title": "Wikipedia:CHECKWIKI/WPC 055 dump",
       "length(body)": 2472791
    },

and https://en.wikipedia.org/wiki/Help:Citation_Style_1/mass_test/fcite_web_ref


In [15]:
import functools

# even with 360 partitions, I get OOM errors
# defaults to 10,000 which is painful on large wikipedia articles
#
# Especially ones like

spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch",1000)  
#spark.conf.set("",5)
# 3% MEM at 125/341

df1 = spark.sql("select * from wikipedia_bronze")

# Tricky workaround for memory leak in the UDFs defined above.
#
# Run against all documents in Wikipedia, it will leak (not use.. unnecessarily grow by)
# about 40GB.  If spark splits this among 10 executors, each will grow by about 4GB, which
# still exceeds the 32GB cheap cluster I'm using.   By splitting the dataframe into 10 batches,
# it kills the "python3 -m pyspark.daemon" workers after each 10th of the data (so never
# leaking over 4GB).  Using .repartition() won't have the same benefits because it seems 
# the task re-uses the "python3 -m pyspark.daemon" for multiple partitions.
num_batches = 10
df2s = [(df1.filter(f'((xxhash64(title)&9223372036854775807) % {num_batches}) = {i}')
            .withColumn('parsed_elements',extract_mediawiki_info_udf('body')))
        for i in range(num_batches)
       ]
df2 = functools.reduce(lambda x,y: x.union(y),df2s)
df3 = df2.withColumn('parsed_elements',psf.from_json('parsed_elements',parsed_schema))
df4 = df3.select('title','body','parsed_elements.*')


In [16]:
# This will take about 3 hours on an 8-core (16-vcore) cluster; about 1 hour on a 48-vcore cluster.
reparse_wikimedia_markup = False
if reparse_wikimedia_markup:
    spark.sql("drop table if exists wikipedia_silver_structured_templates")
    df4.write.format('delta').saveAsTable('wikipedia_silver_structured_templates')

## Spot check structured wikipedia data

At this point we can query the infoboxes, coordinates, etc.

In [17]:
spark.sql('select count(*),count(distinct title) from  wikipedia_silver_structured_templates').collect()

[Row(count(1)=21108360, count(DISTINCT title)=21108326)]

In [18]:
spark.sql('''
  select title as wikipedia_pages_with_multiple_coordinates,
         size(coords) as num_coords 
  from wikipedia_silver_structured_templates 
  order by size(coords) desc 
  limit 10
''').show(100,100)

+------------------------------------------------------------------+----------+
|                         wikipedia_pages_with_multiple_coordinates|num_coords|
+------------------------------------------------------------------+----------+
|Wikipedia:WikiProject National Register of Historic Places/coordsH|      3692|
|                     Sortable list of islands of Western Australia|      1139|
|                                          List of schools in Perak|      1103|
|                                      List of lakes of New Zealand|      1082|
|                                     List of places in Colorado-02|      1077|
|                                        List of places in Colorado|      1031|
|                  List of cities, towns and villages in Gelderland|       842|
|                                 List of waterfalls of Nova Scotia|       832|
|       List of cities, towns and villages in Limburg (Netherlands)|       770|
|                                      L

In [17]:
df = spark.sql('select title,_error_ from wikipedia_silver_structured_templates where _error_ is not null')
print(f'{df.count()} parsing errors')
# TODO - fix these later
# for row in df.take(100): print(json.dumps(row.asDict(True)))
df.show(20,80)

75739 parsing errors
+---------------------------------------------------------+------------------------------------------------+
|                                                    title|                                         _error_|
+---------------------------------------------------------+------------------------------------------------+
|        2021 FIVB Volleyball Boys' U19 World Championship|Object of type Wikicode is not JSON serializable|
|                                           Robert Kallman|        could not convert string to float: 'LAT'|
|                             Totonicapán Uprising of 1820|                                             '1'|
|Wikipedia:Administrators' noticeboard/IncidentArchive1049|Object of type Wikicode is not JSON serializable|
|                         HPV Prevention and Control Board|        could not convert string to float: 'LAT'|
|  Henry J. Carter Specialty Hospital and Nursing Facility|                                             '1'

In [18]:
debug_large_articles = False
if debug_large_articles:
    sizes = spark.sql('''
     select title,length(body) 
       from wikipedia_silver_structured_templates 
      where title not like 'Wikipedia:%' 
        and title not like 'Template:%' 
        and title not like 'Module:%' 
        and title not like 'MediaWiki:%'
        and title not like 'Template:%' 
        and title not like 'Draft:%' 
        and title not like 'Help:%' 
        order by length(body) desc limit 10
     ''').cache()
    sizes.show(40,40)

In [3]:
spark.sql('''select count(*) as articles_with_coords,sum(size(coords)) as num_coords
    from wikipedia_silver_structured_templates 
    where size(coords) > 0
''').show()

+--------------------+----------+
|articles_with_coords|num_coords|
+--------------------+----------+
|             1203242|   1855143|
+--------------------+----------+



In [5]:
# About a half million h6 grids with information in wikipedia.
spark.sql('''
  select count(distinct coord.h3[3]) as num_h3,
         count(distinct coord.h3[4]) as num_h4,
         count(distinct coord.h3[5]) as num_h5,
         count(distinct coord.h3[6]) as num_h6,
         count(distinct coord.h3[7]) as num_h7,
         count(distinct coord.h3[8]) as num_h8,
         count(distinct coord.h3[9]) as num_h9,
         count(*)
 from wikipedia_silver_structured_templates
 lateral view explode(coords) as coord
''').toPandas()

,num_h3,num_h4,num_h5,num_h6,num_h7,num_h8,num_h9,count(1)
0,21799,72608,206136,498461,883727,1164699,1335738,1855143


TODO

* http://mcburton.net/blog/static-files/

In [26]:
spark.sql(r"select title,replace(substr(body,1,40)||'...','\n','\\n') as body from wikipedia_bronze limit 5").show(5,False)

+-----------------------------+---------------------------------------------+
|title                        |body                                         |
+-----------------------------+---------------------------------------------+
|Black panther                |{{short description|Melanistic colour va...  |
|Great white shark            |{{other uses of|great white|Great White ...  |
|Melisende, Queen of Jerusalem|{{short description|Queen regnant of the...  |
|Carcharodon carcharias       |#REDIRECT [[Great white shark]] \n \n {{Re...|
|Method                       |{{Wiktionary|method}} \n '''Method''' ({{... |
+-----------------------------+---------------------------------------------+



In [21]:
from IPython.display import FileLink, FileLinks
FileLink('demo.html')

/home/jovyan/work/wikipedia_in_spark/notebooks/demo.html